In [1]:
%cd ../

/src


In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.metrics import roc_auc_score
import torch
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

%matplotlib inline

In [3]:
import os

In [4]:
from zope.dottedname.resolve import resolve

from src.datasets.MNIST import MNIST
from src.models.DeepADoTS.autoencoder import AutoEncoder
from src.models.DeepADoTS.dagmm import DAGMM
from src.models.DeepADoTS.rnn_ebm import RecurrentEBM

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
print(torch.cuda.is_available())

True


In [6]:
def prepare_dataset(class_name='MNIST.MNIST', train_num=None, test_num=None):
    class_name_head = "src.datasets"
    class_name = class_name_head + '.' + class_name
    x_train, y_train, x_test, y_test = resolve(class_name)(seed=0).data()
    # Use fewer instances for demonstration purposes
    if train_num is not None:
        x_train, y_train = x_train[:train_num], y_train[:train_num]
    if test_num is not None:
        x_test, y_test = x_test[:test_num], y_test[:test_num]
    return x_train, y_train, x_test, y_test

def exec_autoencoder(x_train, x_test):
    model = AutoEncoder(sequence_length=1, num_epochs=40, hidden_size=10, lr=1e-4, gpu=0)
    model.fit(x_train)
    output = model.predict_val(x_test)
    return model, output

def exec_dagmm(x_train, x_test):
    model = DAGMM(sequence_length=1, num_epochs=40, hidden_size=10, lr=1e-4, gpu=0)
    model.fit(x_train)
    print('end_fit')
    output = model.predict(x_test)
    return output

def exec_rnn(x_train, x_test):
    model = RecurrentEBM(gpu=0)
    model.fit(x_train)
    print('end_fit')
    output = model.predict(x_test)
    return output

In [7]:
print(torch.cuda.is_available())
x_train, y_train, x_test, y_test = prepare_dataset(train_num=1000, test_num=100)
# model, output = exec_autoencoder(x_train, x_test)
# output=exec_dagmm(x_train, x_test)
# output = exec_rnn(x_train, x_test)
# print(output)

True
11493376/11490434 [==============================] - 0s 0us/step


## check layers

In [8]:
from torchsummary import summary

In [9]:
n_features = x_train.shape[1]

In [40]:
## AutoEncoder
from src.models.DeepADoTS.autoencoder import AutoEncoderModule
model = AutoEncoderModule(n_features=n_features, sequence_length=1,hidden_size=10, seed=1, gpu=0)

In [41]:
print(model)

AutoEncoderModule(
  (_encoder): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): Tanh()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): Tanh()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): Tanh()
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): Tanh()
    (8): Linear(in_features=64, out_features=32, bias=True)
    (9): Tanh()
    (10): Linear(in_features=32, out_features=10, bias=True)
  )
  (_decoder): Sequential(
    (0): Linear(in_features=10, out_features=32, bias=True)
    (1): Tanh()
    (2): Linear(in_features=32, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): Tanh()
    (6): Linear(in_features=128, out_features=256, bias=True)
    (7): Tanh()
    (8): Linear(in_features=256, out_features=512, bias=True)
    (9): Tanh()
    (10): Linear(in_features=512, out_features=784, bias=True)
  )
)


In [44]:
summary(model, input_size=(1, n_features), batch_size=10, device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [10, 512]         401,920
              Tanh-2                  [10, 512]               0
            Linear-3                  [10, 256]         131,328
              Tanh-4                  [10, 256]               0
            Linear-5                  [10, 128]          32,896
              Tanh-6                  [10, 128]               0
            Linear-7                   [10, 64]           8,256
              Tanh-8                   [10, 64]               0
            Linear-9                   [10, 32]           2,080
             Tanh-10                   [10, 32]               0
           Linear-11                   [10, 10]             330
           Linear-12                   [10, 32]             352
             Tanh-13                   [10, 32]               0
           Linear-14                   

In [45]:
from torch.utils.tensorboard import SummaryWriter

In [46]:
# Writer will output to ./runs/ directory by default
writer = SummaryWriter()

In [47]:
x_train_tensor = torch.from_numpy(x_train.values)
device = torch.device('cuda:0')
x_train_tensor = x_train_tensor.to(device)

In [48]:
writer.add_graph(model, x_train_tensor)
writer.close()

In [49]:
%load_ext tensorboard

In [50]:
%tensorboard --logdir runs --port 6006 --host 0.0.0.0